In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import mnist

In [17]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [18]:
# Preprocess the data: Normalize and reshape
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


In [19]:
# Add a channel dimension to make it compatible with Conv2D layers
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [20]:
# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [21]:
# Define a Sequential model using Keras
model = models.Sequential()

In [22]:
# Add layers to the model
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [23]:
# Flatten the output of the previous layers to feed into Dense layers
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))


In [24]:
# Output layer: 10 classes with softmax activation
model.add(layers.Dense(10, activation='softmax'))


In [25]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [30]:
# Define callbacks for early stopping and saving the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)


In [31]:
# Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=64,
                    validation_split=0.2,
                    callbacks=[early_stopping, model_checkpoint])


Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 47s 60ms/step - accuracy: 0.7956 - loss: 0.6306 - val_accuracy: 0.9813 - val_loss: 0.0648
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 46s 61ms/step - accuracy: 0.9743 - loss: 0.0886 - val_accuracy: 0.9847 - val_loss: 0.0525
Epoch 3/20
278/750 ━━━━━━━━━━━━━━━━━━━━ 26s 56ms/step - accuracy: 0.9844 - loss: 0.0553

KeyboardInterrupt: 

In [32]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9836 - loss: 0.0486
Test accuracy: 0.9861999750137329


In [33]:
# Save the final model
model.save('mnist_keras_model.keras')